In [1]:
import sys
import os
import pandas as pd

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h')
print(f"{current_date}")

2025-03-04 06:00:00


In [3]:
current_date

Timestamp('2025-03-04 06:00:00')

In [4]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

In [5]:
fetch_data_to

Timestamp('2025-03-04 06:00:00')

In [6]:
fetch_data_from

Timestamp('2025-02-04 06:00:00')

In [7]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [8]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [9]:
rides

,pickup_datetime,pickup_location_id
809040,2025-02-12 16:25:44,2
1115741,2025-02-15 16:56:40,2
1058,2025-02-04 06:08:40,3
68810,2025-02-04 18:17:57,3
97325,2025-02-05 05:47:46,3
...,...,...
2845645,2025-03-04 05:50:32,263
2884813,2025-03-04 05:53:18,263
2884839,2025-03-04 05:56:00,263
2845636,2025-03-04 05:58:34,263


In [10]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [11]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-04 06:00:00,2,0
1,2025-02-04 07:00:00,2,0
2,2025-02-04 08:00:00,2,0
3,2025-02-04 09:00:00,2,0
4,2025-02-04 10:00:00,2,0
...,...,...,...
168667,2025-03-04 01:00:00,263,6
168668,2025-03-04 02:00:00,263,6
168669,2025-03-04 03:00:00,263,3
168670,2025-03-04 04:00:00,263,8


In [12]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168672 entries, 0 to 168671
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         168672 non-null  datetime64[ns]
 1   pickup_location_id  168672 non-null  int16         
 2   rides               168672 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 1.9 MB


In [14]:
from dotenv import load_dotenv
import os

# Explicitly load the .env file
load_dotenv(dotenv_path="/Users/lohithvattikuti/Desktop/sp25_taxi-main/.env")

# Retrieve values from environment variables
project_name = os.getenv("HOPSWORKS_PROJECT_NAME")
api_key = os.getenv("HOPSWORKS_API_KEY")
feature_group_name = os.getenv("FEATURE_GROUP_NAME")
feature_group_version = os.getenv("FEATURE_GROUP_VERSION")

# Debug: Print the loaded values to ensure they are loaded correctly
print("HOPSWORKS_PROJECT_NAME:", project_name)
print("HOPSWORKS_API_KEY:", api_key)
print("FEATURE_GROUP_NAME:", feature_group_name)
print("FEATURE_GROUP_VERSION:", feature_group_version)

# Check if any environment variable is missing
if not project_name or not api_key or not feature_group_name or not feature_group_version:
    raise ValueError("One or more required environment variables are missing.")

# Convert FEATURE_GROUP_VERSION to an integer (if needed)
feature_group_version = int(feature_group_version)

# Connect to Hopsworks project
import hopsworks

project = hopsworks.login(
    project=project_name,
    api_key_value=api_key
)

print(f"Successfully connected to Hopsworks project: {project_name}")

# Connect to Feature Store
feature_store = project.get_feature_store()

# Connect to Feature Group
feature_group = feature_store.get_feature_group(
    name=feature_group_name,
    version=feature_group_version,
)

print(f"Successfully connected to Feature Group: {feature_group_name} (version {feature_group_version})")


HOPSWORKS_PROJECT_NAME: lohithvattikuti
HOPSWORKS_API_KEY: xBEuFPB7sUswEa06.vcqv2NPHG9HzWLhfuUtKfARRCjFrXj9G7zwwegSZtVSTIV1ksU5tQzp3xRIuGwHO
FEATURE_GROUP_NAME: time_series_hourly_feature_group
FEATURE_GROUP_VERSION: 1
2025-03-04 01:07:40,367 INFO: Initializing external client
2025-03-04 01:07:40,368 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 01:07:41,283 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214690
Successfully connected to Hopsworks project: lohithvattikuti
Successfully connected to Feature Group: time_series_hourly_feature_group (version 1)


In [ ]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 168672/168672 | Elapsed Time: 00:25 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214680/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)